# Inheritance

In [1]:
// Class inherited using extends keyword
class Base {
    
}

class Child extends Base {
    
}


defined class Base
defined class Child

* Adding `final` keyword to a class prevents it from being extended
* Methods and fields can also be declared final to prevent overriding.

In [2]:
// non abstract methods require override keyword
class Base {
    def greet(name: String): Unit = {
        println(s"Hello, $name")
    }
}

class Derived extends Base {
    override def greet(name: String): Unit = {
        println(s"Hi, $name")
    }
}

defined class Base
defined class Derived

To invoke methods of the parent class, we use `super` keyword.

## Type checks and casts

* `isInstanceOf` and `asInstanceOf` are used for type checking and casting
respectively. `isInstanceOf` returns true if the even if a subclass
instance is checked against superclass

In [3]:
val d = new Derived()
println(d.isInstanceOf[Base])

true


d: Derived = ammonite.$sess.cmd1$Helper$Derived@32224d7d

In [4]:
// To check against exact class type
d.getClass == classOf[Derived]

res3: Boolean = true

In [5]:
// we can also use pattern matching for type checking
d match {
    case b: Base => b.greet("John")
    case _ => ()
}

Hi, John


* `protected` members are accessible to subclasses

## Superclass construction

* Auxiliary constructors of derived classes should never invoke
the superclass constructor directly. 
This means we cannot use `super` keyword inside auxiliary constructor
to call the superclas constructor. Only the primary constructor
can call the superclass constructor.

In [6]:
class Person(val name: String, val age: Int)

class Employee(name: String, 
               age: Int, 
               val salary : Double) extends Person(name, age)

defined class Person
defined class Employee

## Overriding Fields

* val or a parameterless def in base can be overridden with a val in derived
* parameterized def can be overridden only with def
* var can override only abstract var.

**NOTE**: We should be very cautious when exposing a var
as a public field, because then we are stuck with the compiler
generated getter and setter in all subclasses. If we want to
customize the setter logic, then we are in trouble because we cannot
override a var that's not abstract in the base.

## Anonymous subclasses

In [7]:
val student = new Person("John", 25) {
    def greet: Unit = println(s"Hello, $name")
}
student.greet

Hello, John


student: Person{def greet: Unit} = ammonite.$sess.cmd6$Helper$$anon$1@6c4d2a19

## Abstract classes

* classes made abstract using `abstract` keyword. Abstract classes
cannot be instantiated.
* Abstract classes have fields or methods declaration but no definition.
* To override abstract members, `override` keyword is not required.
* We can create anonymous subclasses from abstract base classes by
providing implementation for all abstract members.

In [8]:
// abstract fields are uninitialized
abstract class SimpleClass {
    //abstract field
    val id: Int

    // abstract field
    var name: String
    
    //abstract method
    def description: String
}

defined class SimpleClass

## Construction Order

* Superclass constructor is executed before the subclass one.
* We can debug construction order using `-Xcheckinit` compiler flag.
* Using **early definition** we can initialize val fields in subclass before
the superclass.

## Scala type hierarchy

* Any is the root type
* AnyVal is the root of all value types(primitives in Java)
* AnyRef is the base of all reference types
* Null type has only instance which is null. Null is a subtype
of all reference types.
* Nothing type has no instance. Nothing is a subtype of all types.

## Object equality

* Override the `equals` method to compare equality of two objects
* To compare reference of the two objects, use the `eq` method 
provided by `AnyRef`. This is similar to `is` operator in python.

In [12]:
// eq method on AnyRef is used to check if the object
// references are the same
// As we know null is the only instance of Null type 
// we can check like this below
println(null.eq(null))

val obj1 = "Hello"
val obj2 = "Hello" // performs compile time optimization
val obj3 = obj1
val obj4 = List[Char]('H', 'e', 'l', 'l', 'o').mkString
println(s"obj1.eq(obj2) -> ${obj1.eq(obj2)}")
println(s"obj1.eq(obj3) -> ${obj1.eq(obj3)}")
println(s"obj1.eq(obj4) -> ${obj1.eq(obj4)}")

true
obj1.eq(obj2) -> true
obj1.eq(obj3) -> true
obj1.eq(obj4) -> false


import scala.util.Random

// eq method on AnyRef is used to check if the object
// references are the same
// As we know null is the only instance of Null type 
// we can check like this below

obj1: String = "Hello"
obj2: String = "Hello"
obj3: String = "Hello"
obj4: String = "Hello"

In [15]:
// We define equals and hashCode together
// so that we could use the hashCode for comparison
// two objects with same hashcode can be treated as equal
class Item(val name: String, val price: Double) {
    
    // parameter type should be Any because we
    // are overriding equals method provided by Any
    // Its not recommended to have a method named ==
    override def equals(that: Any): Boolean = {
        var other: Option[Item] = None
        if (that.isInstanceOf[Item]) {
            other = Some(that.asInstanceOf[Item])
        }
        
        other.isDefined && (this.name == other.get.name && 
                            this.price == other.get.price)
    }
    
    
    // 1. hashCode should use the same fields used for equality
    // 2. ## is a null safe version.
    override def hashCode = (name, price).##
}

val item1 = new Item("a", 100.0)
val item2 = new Item("a", 100.0)
val item3 = new Item("b", 10.0)

println(item1 == item2)
println(item1.equals(item2))
println(item1 == item3)

true
true
false


defined class Item
item1: Item = ammonite.$sess.cmd14$Helper$Item@1161dc44
item2: Item = ammonite.$sess.cmd14$Helper$Item@1161dc44
item3: Item = ammonite.$sess.cmd14$Helper$Item@65d5d92f

In [16]:
// returns 0 as the hash for null
null.##

res15: Int = 0

## Value types

* custom value types can be created by extending AnyVal
* Primary constructor should have only one parameter.
* The class should not have any other fields or constructor, but it can
provided parameterless and parameterized methods.
* `equals` and `hashcode` are automatically provided.
* Class can implement only those traits that explicitly extends Any

In [16]:
// This snippet won't work on this almond scala kernel
// You might need to execute this in a separate scala src file

class StockNos(val units: Int) extends AnyVal {
    // providing method for multiplication
    def *(other: Int): StockNos = new StockNos(this.units * other)
    
    override def toString = s"StockNos(${this.units})"
}

val stockNos = new StockNos(5)
println(stockNos * 2)